In [ ]:
import calendar
import datetime
from datetime import timedelta
startDate = datetime.strptime("20220723", "%Y%m%d")

date = startDate + timedelta(days=1)
year= currentdate.year
month = currentdate.month
day = currentdate.day

#系统默认：星期一作为一周的第一天（即：0），星期日作为一周的最后一天（即：6）
print(calendar.weekday(year,month,day))
currentday =calendar.weekday(year,month,day)


In [ ]:
run erfpipe.py
